In [1]:
import os

base_path = '/Users/yuta/git/github.com/tuki0918/ksnctf-34-are-you-human/resources/image/'

files = os.listdir(base_path)
files_file = [f for f in files if os.path.isfile(os.path.join(base_path, f))]
# print(files_file)   # ['file1', 'file2.txt', 'file3.jpg']
print(len(files_file))

7571


In [2]:
%matplotlib inline

import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
model = load_model('model-v0f-3ch.h5')
model.summary()

train_data_dir = '/Users/yuta/git/github.com/tuki0918/ksnctf-34-are-you-human/resources/dataset-manual-3ch/'

img_height = 32
img_width = 32
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
#     color_mode='grayscale',
    target_size=(img_height, img_width),
    batch_size=batch_size)

labels = dict((v, k) for k, v in train_generator.class_indices.items())

def prepare_xz(img, debug=False):
    img_rows, img_cols = 32, 32
    re_img = cv2.resize(img, (img_rows, img_cols))
    re_img = re_img.astype('float32')
    re_img /= 255
    x = re_img.reshape(1, img_rows, img_cols, 3)
    return x

def predict_xz(img, debug=False):
    x = prepare_xz(img, debug)
    x_proba = model.predict(x)
    x_classes = x_proba.argmax(axis=-1)
    return x_classes[0]

def pppz(path, debug=False):
    img = cv2.imread(path)

    if (debug):
        plt.figure(figsize=(30,15))
        plt.imshow(img)

    result = []
    for i in range(32):

        x = (32*i)
        y = 0
        w = 32
        h = 32

        try:
            dist = img[y: y + h, x: x + w]
            xz = predict_xz(dist, debug)
            result.append(labels[xz])
        except Exception as e:
            print(path, e)
            result.append('*')
    return result

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12544)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1605760   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [6]:
import numpy as np
import pandas as pd

matrix = []

for img_path in files_file:

    try:
        img_name, img_ext = os.path.splitext(img_path)
        img_name_10num = int(img_name, 16)
        result = pppz(os.path.join(base_path, img_path), debug=False)
        result_code = ''.join(result)
        record = [img_name, img_name_10num, result_code]
        matrix.append(record)
    except Exception as e:
        print(e)

print(matrix.shape)

AttributeError: 'list' object has no attribute 'shape'

In [7]:
matrix = np.array(matrix)
print(matrix.shape)

df = pd.DataFrame(matrix, columns=['name', 'num', 'result'])
df.head()

(7571, 3)


,name,num,result
0,0000da40,55872,13AF2F6E000CED34503DB8D697443B6A
1,0000e580,58752,3DD9E0425F335D7CD294CF0E0435359B
2,00013aa0,80544,82C69F2EFB6B06C66EF193F1B6DB5490
3,00002940,10560,08E1215816763408B99C79FD474253A1
4,0000a080,41088,CDDDBED7962F62AE18C83779ABEAF5B3


In [20]:
df['num2'] = df['num'].astype(int)
df_s = df.sort_values('num2', ascending=True)
df_s.head()

,name,num,result,num2
1674,00000000,0,FFD8FFE1001845786966000049492800,0
237,00000010,16,080000000000000000000D00FFEC0011,16
7308,00000020,32,4475636B190001000400D0003C0000FF,32
5914,00000030,48,EE000E41646F62650064C000000001FF,48
4933,00000040,64,DB0084000604040D0504060505060906,64


In [21]:
df_s.tail()

,name,num,result,num2
2443,0001d8e0,121056,4DFD5F48D85E627437F96B5EAEDE31BA,121056
4782,0001d8f0,121072,7A389C7588F3938F2D217E36F2665490,121072
6487,0001d900,121088,23953A97522E3E3D3998487AB9F15768,121088
6755,0001d910,121104,A652AF85BD5D9E4CFEFFE21E20B2A9B8,121104
596,0001d920,121120,689C1A88D08203F197F1411FFFF14FFF,121120


In [23]:
b_file = open('flag.jpg', 'w')

for index, row in df_s.iterrows():
    b_file.write(row.result)

b_file.flush()
b_file.close()